In [ ]:
import sys
import os

from dd_widgets import timeseries as ts
from dd_widgets.timeseries.nbeats import NBEATS

In [ ]:
labels = ["0"]
datadir = "/opt/platform/data/nasa"
model_name = "nbeats_nasa"

nbeats = NBEATS(
    sname = model_name,
    host = "deepdetect",
    port = 8080,
    
    model_repo = "/opt/platform/models/training/" + model_name,
    training_repo = datadir + "/train",
    testing_repo = datadir + "/test",
    # NBEATS predicts the future from the past: target columns are also input columns
    label_columns = ["0"],
    # ignored columns. The model won't predict the future of these signals
    ignore_columns = [],
    
    # Model parameters
    # number of past steps taken as input
    backcast_timesteps = 100,
    # number of future steps predicted
    forecast_timesteps = 100,
    # NBEATS hyperparameters
    # 2 x generic stack with theta = 128, 5 blocks per stacks, hidden unit size of 128 everywhere
    # more details here https://www.deepdetect.com/server/docs/api/#create-a-service and in NBEATS paper
    template_params = {
        "stackdef": ["g512", "g512", "b5", "h512"],
        "backcast_loss_coeff" : 1.0
    },

    # Training parameters
    gpuid = 2,
    base_lr = 0.001,
    batch_size = 100,
    iter_size = 1,
    test_interval = 2000,
    iterations = 100000,
    offset = 1,
    
    # Predict parameters
    pred_interval = 1,
    autoregressive = False,
)

# compute predictions / reload if necessary
plots = ts.TimeseriesPlot(
    models = [nbeats],
    # Target columns (available for prediction in the selector)
    target_cols = labels,
    ignored_cols = [],
    # Root directory for all dataset files
    datadir = datadir,
    # Each dataset file path relative to the root directory
    datafiles = ts.get_datafiles(datadir),
    # Cache directory for prediction. Predictions can be saved here and reused later.
    output_dir = "/opt/platform/data/predictions/",
)
# Display dataset
plots.dataset_ui()

In [ ]:
# Training
nbeats

In [ ]:
# Delete train service
nbeats.delete()

# Change the path of the model after publishing
# nbeats.model_repo.value = "/opt/platform/models/private/" + model_name

# Display forecasting results
plots.forecast_ui(use_cached = True)

In [ ]:
# Learn error distribution on train set.
plots.predict_all(override = False)
plots.learn_anomalies(nbeats, ["train/A-4.csv"])

# Display detected anomalies with associated error
plots.anomalies_ui(use_cached = True)